<center>
<a href="https://github.com/kamu-data/kamu-cli">
<img alt="kamu" src="https://raw.githubusercontent.com/kamu-data/kamu-cli/master/docs/readme_files/kamu_logo.png" width=270/>
</a>
</center>

<br/>

<div align="center">
<a href="https://github.com/kamu-data/kamu-cli">Repo</a> | 
<a href="https://docs.kamu.dev/cli/">Docs</a> | 
<a href="https://docs.kamu.dev/cli/learn/learning-materials/">Tutorials</a> | 
<a href="https://docs.kamu.dev/cli/learn/examples/">Examples</a> |
<a href="https://docs.kamu.dev/cli/get-started/faq/">FAQ</a> |
<a href="https://discord.gg/nU6TXRQNXC">Discord</a> |
<a href="https://kamu.dev">Website</a>
</div>

<center>

# 3. Alternative Reality Pipeline

</center>

<div class="alert alert-block alert-info">
If you skipped the previous chapter or continuing after a break - use the following commands to get your environment ready for this chapter:
    
<p style="background:black">
<code style="background:black;color:white">cd "02 - Web3 Data (Ethereum trading example)"
./init-chapter-3.sh
</code>
</p>
</div>

# Extending our Use Case

In the first chapter we've built a pipeline that tracks the market value of our token investments:

![x](files/pipeline-1.png)

But it would be great not only see the current state, but to also compare the current outcome to some alternative investments that *we could've made*.

In this chapter we will extend our pipeline to provide an "alternate reality" view of **investing same amount of money into the S&P 500** market index.

# Getting the SPY ticker data
The last root dataset we need for this demo is the `SPY` ticker data - an ETF that tries to replicate the performance of the S&P 500.

We will source this data from [Alpha Vantage API](https://www.alphavantage.co/) using their free tier. You can find the dataset manifest [here](https://github.com/kamu-data/kamu-contrib/tree/master/co.alphavantage).

<div class="alert alert-block alert-success">

But we will simply pull it from IPFS:

<p style="background:black">
<code style="background:black;color:white">kamu pull "ipns://co.alphavantage.tickers.daily.spy.ipns.kamu.dev" --as co.alphavantage.tickers.daily.spy
</code>
</p>
</div>

<div class="alert alert-block alert-warning">
Pulling from IPFS may take a few minutes, so if you'd like to sacrifice the "full decentralized data experience" for speed you can also pull data from Kamu Node:
    
<p style="background:black">
<code style="background:black;color:white">kamu pull ${KAMU_NODE_URL}kamu/co.alphavantage.tickers.daily.spy
</code>
</p>
</div>

In [ ]:
%load_ext kamu
import kamu
import pandas as pd
import hvplot.pandas
import holoviews as hv

pd.set_option('max_colwidth', None)

con = kamu.connect("file://")

In [ ]:
%%sql
select 
    event_time, close
from "co.alphavantage.tickers.daily.spy"
where event_time > '2010-01-01'

--## Switch to "Area" viz type to view results

# Alternative portfolio dataset
Remember `account.tokens.portfolio.usd` dataset?

<div class="alert alert-block alert-info">
To remind yourself how it looks like use:
<p style="background:black">
<code style="background:black;color:white">kamu inspect schema account.tokens.portfolio.usd
kamu tail account.tokens.portfolio.usd
</code>
</p>
</div>

It already has `eth_amount_as_usd` column that at the time of a transaction tells us what was the market value of `ETH` used to purchase or sell the tokens.

By combining this column with the stock ticker we can create a dataset that makes imaginary purchases of the `SPY` ETF for the same amount.

<div class="alert alert-block alert-success">
Add the prepared dataset and pull data:
<p style="background:black">
<code style="background:black;color:white">kamu add datasets/account.whatif.reth-vs-snp500.portfolio.yaml
kamu pull account.whatif.reth-vs-snp500.portfolio
</code>
</p>
</div>

This dataset essentially enriches the existing `portfolio` dataset with two columns: `alt_spy_amount` (per transaction) and `alt_spy_balance` (cumulative).

Have a look at the manifest file yourself if you're interested, but there's nothing there that we haven't done before.

# Alternative market value dataset
Now, just like before, we will JOIN the alternative portfolio back onto the ticker data to get the market value of our imaginary stocks over time.

<div class="alert alert-block alert-success">
Add the prepared dataset and pull data:
<p style="background:black">
<code style="background:black;color:white">kamu add datasets/account.whatif.reth-vs-snp500.market-value.yaml
kamu pull account.whatif.reth-vs-snp500.market-value
</code>
</p>
</div>

Drum roll... Here comes the exciting part - let's compare the two investments!

In [ ]:
%%sql -o portfolio -q
select * from "account.tokens.portfolio.usd"
where token_symbol = 'rETH'

In [ ]:
%%sql -o market_value -q
select * from "account.tokens.portfolio.market-value"

In [ ]:
%%sql -o alternative_market_value -q
select * from "account.whatif.reth-vs-snp500.market-value"

In [ ]:
max_height = max(
    alternative_market_value["alt_spy_market_value_usd"].max(),
    market_value["token_market_value_usd"].max(),
    market_value["token_book_value_eth_as_usd"].max(),
)

alternative_market_value.hvplot.line(
    x="event_time", 
    y=["alt_spy_market_value_usd"],
    xlabel="Time",
    ylabel="USD",
    label="alt_spy_market_value_usd",
    title="rETH vs S&P500",
    height=500, 
    width=800,
) * market_value.hvplot.line(
    x="event_time", 
    y=["token_market_value_usd", "token_book_value_eth_as_usd"],
    legend="bottom_right",
) * hv.Spikes(
    portfolio["block_time"]
).opts(
    line_dash="dashed", 
    color="green", 
    line_width=.2,
    line_alpha=0.7, 
    spike_length=max_height
)

As of the time of this writing, the best investment option by far was to keep your cash under the mattress :)

<div class="alert alert-block alert-warning">

Have to remind you that this is not meant to express any opinion on cryptocurrency or be taken as an investing advise. 
    
We are here purely to **give you the tools to make your own data-driven decisions**.

</div>

<div class="alert alert-block alert-info">

Quick reminder that the final version of the pipeline is also available in Kamu Node:
    
${KAMU_WEB_UI_URL}kamu/account.tokens.portfolio.usd?tab=lineage

![](files/kamu-node.png)

</div>

# Conclusion

<mark>Just how cool this last graph is?!</mark>

Unlike most brokerages that only show you whether you're "up" or "down", we can:
- Clearly see the performance of our portfolio over time
- See how good or badly timed our investments were
- Can set up a wide range of alternate investments to have more reference points to analyze performance

But more importantly:
- We **created all this with code** - using a fairly low complexity Streaming SQL
    - Certainly much less complex and error prone than its Batch equivalent
- We got our data from **blockchain**, **decentralized sources** like IPFS, and from conventional APIs
- And processed it all on **our computer**, without the need of some beefy cloud data warehouse with a thousand $/m subscription.

Hopefully this demo shows you the utility of `kamu` even at this early stage, but also our vision:
- Where closer integration with blockchains can make them as easy to query as any other ODF dataset
- Where large decentralized data pipelines can process data at real-time speeds and supply it to Smart Contracts
- Where the boundary between on- and off-chain data is erased

[Join us on Discord](https://discord.gg/nU6TXRQNXC) and tell us what you think!